In [1]:
import pandas as pd
import sklearn.preprocessing as sklpre
import sklearn.model_selection as sklms
import sklearn.metrics as sklmet
import matplotlib.pyplot as plt
import numpy as np
import keras


Using TensorFlow backend.


In [2]:
import pandas as pd
import sklearn.preprocessing as sklpre
import sklearn.model_selection as sklms

import numpy as np



def load_wine_split(wine=1):
    if wine==1:
        data="winequality-white.csv"
    elif wine==2:
        data="winequality-white.csv",
    else:
        exit('Program aborted. Provide wine type 1 (white) or 2 (red) in function load_wine_split')
    df = pd.read_csv(data, sep=";")
    X = df.drop("quality", axis=1)
    y = df["quality"]

    # stratified split
    X_train, X_test, y_train, y_test = sklms.train_test_split(
        X, y, stratify=y, test_size=0.33
    )

    # pre-process data
    scaler = sklpre.MinMaxScaler()  # alternative: StandardScaler()
    encoder = sklpre.OneHotEncoder(categories="auto", sparse=False)  # onehot-encoder

    # Scale design matrix (no categorical)
    X_train = pd.DataFrame(
        scaler.fit_transform(X_train.to_numpy()), columns=X_train.columns
    )  # to_numpy not needed?

    X_test = pd.DataFrame(
        scaler.fit_transform(X_test.to_numpy()), columns=X_test.columns
    )  # to_numpy not needed?

    y_train  = pd.DataFrame(
        encoder.fit_transform(y_train.to_numpy().reshape(-1, 1)),
        columns=encoder.categories_,
    )

    y_test = pd.DataFrame(
        encoder.fit_transform(y_test.to_numpy().reshape(-1, 1)), columns=encoder.categories_
    )

    return X_train,X_test,y_train,y_test


from keras import optimizers as kerasopt
from keras import regularizers as kerasreg
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout

def create_model(
    nl1=0,
    nl2=1,
    nl3=0,
    nn1=12,
    nn2=11,
    nn3=10,
    lr=0.01,
    decay=0.0,
    l1=0.01,
    l2=0.01,
    act="relu",
    dropout=0,
    input_shape=11,
    output_shape=7, #synes jo ikke på putput!
):
    # set optimizer
    opt = kerasopt.Adam(lr=lr, beta_1=0.9, beta_2=0.999, decay=decay) #decay?
    # set regularizers
    reg = kerasreg.l1_l2(l1=l1, l2=l2)

    model = Sequential()
    model.add(
        Dense(nn1, input_dim=input_shape, activation=act, kernel_regularizer=reg) #dropout her?
    )  # first layer
    if dropout != 0:
        model.add(Dropout(dropout))
    for i in range(nl1):  # add up to nl1 layers with nn1 nodes
        model.add(Dense(nn1, activation=act, kernel_regularizer=reg))
        if dropout != 0:
            model.add(Dropout(dropout)) #hva er egentlig dropout? Hvor lav terskel for å fjerne nevroner?

    for i in range(nl2):  # add up to nl2 layers with nn2 nodes
        model.add(Dense(nn2, activation=act, kernel_regularizer=reg))
        if dropout != 0:
            model.add(Dropout(dropout))

    for i in range(nl3):  # add up to nl3 layers with nn3 nodes
        model.add(Dense(nn3, activation=act, kernel_regularizer=reg))
        if dropout != 0:
            model.add(Dropout(dropout)) #dropout her?
    model.add(Dense(output_shape, activation="softmax"))  # ouput layer
    if dropout != 0:
        model.add(Dropout(dropout))
    model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])
    return model


In [3]:
# load data
np.random.seed(3)
X_train,X_test,y_train,y_test=load_wine_split(wine=1)

In [4]:
#set parameters
lr = [1e-2, 1e-3]  # , 1e-4] # learning rate
decay = [1e-6, 1e-9]  # ,0] # deacy
activation = ["relu", "sigmoid"] # activation

nl1 = [0, 1]  # ,2] # numbers of layers of type nl1
nl2 = [0, 1]  # ,2] # numbers of layers of type nl2
nl3 = [0]  # ,1]#,2] # numbers of layers of type nl3

nn1 = [12]  # ,8]#,20] # neurons in each layer of type nl1
nn2 = [11]  # ,8]#,14] # neurons in each layer of type nl2
nn3 = [10, 6]  # ,9] # neurons in each layer of type nl3

dropout = [0, 0.1, 0.2, 0.3] #dropout
l1 = [0, 0.01, 0.003, 0.001, 0.0001] #Regularizer 1
l2 = [0, 0.01, 0.003, 0.001, 0.0001] #Regularizer 2

dropout = [0.1, 0.2]
l1 = [0]  # ,0.1]
l2 = [0]  # ,0.1]

epochs=[2,3]

param_grid = dict(
    nl1=nl1,
    nl2=nl2,
    nl3=nl3,
    nn1=nn1,
    nn2=nn2,
    nn3=nn3,
    act=activation,
    l1=l1,
    l2=l2,
    lr=lr,
    decay=decay,
    dropout=dropout,
    epochs=epochs
)


In [19]:

from keras import optimizers as kerasopt
from keras import regularizers as kerasreg
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout

In [16]:

grid = sklms.RandomizedSearchCV(
    estimator=model,
    cv=sklms.KFold(3),
    param_distributions=param_grid,
    verbose=1,
    n_iter=10,
    n_jobs=-1,
    scoring='neg_mean_absolute_error'
) #GridSearchCV
grid_result = grid.fit(X_train.values, np.argmax(y_train.values,axis=1))
cv_results_df = pd.DataFrame(grid_result.cv_results_)
cv_results_df.to_csv("gridsearch.csv")

Fitting 3 folds for each of 10 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:   18.3s finished
/home/student/anaconda3/envs/fysstk/lib/python3.7/site-packages/sklearn/model_selection/_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


Epoch 1/3
3281/3281 [==============================] - 1s 363us/step - loss: 2.7038 - accuracy: 0.4410
Epoch 2/3
3281/3281 [==============================] - 1s 272us/step - loss: 2.4435 - accuracy: 0.4861
Epoch 3/3
3281/3281 [==============================] - 1s 291us/step - loss: 2.6229 - accuracy: 0.4858


In [17]:

best_params = grid_result.best_params_
best_model = grid_result.best_estimator_
best_score= grid_result.best_score_
print("Best fit params:", best_params)
print('Best score:', best_score)




Best fit params: {'nn3': 6, 'nn2': 11, 'nn1': 12, 'nl3': 0, 'nl2': 0, 'nl1': 0, 'lr': 0.01, 'l2': 0, 'l1': 0, 'epochs': 3, 'dropout': 0.1, 'decay': 1e-06, 'act': 'relu'}
Best score: -0.5586711368485218


In [18]:

y_pred=best_model.predict(X_train.to_numpy(), verbose=0)
y_train_1=np.argmax(y_train.values,axis=1)


f1_best=mean_absolute_error(y_train_1,y_pred)
print("Best MAD", f1_best)

Best MAD 0.5748247485522706
